# Task 2

### Load Drug dataset

In [37]:
import pandas as pd

drug_dataset = pd.read_csv('../data/drug200.csv')

print(drug_dataset.head())


   Age Sex      BP Cholesterol  Na_to_K   Drug
0   23   F    HIGH        HIGH   25.355  drugY
1   47   M     LOW        HIGH   13.093  drugC
2   47   M     LOW        HIGH   10.114  drugC
3   28   F  NORMAL        HIGH    7.798  drugX
4   61   F     LOW        HIGH   18.043  drugY


In [4]:
import matplotlib.pyplot as plt

sp = drug_dataset['Drug'].value_counts().plot(title='Drug instance count', figsize=(18, 6),
                                         kind='barh')

plt.savefig('../out/drug-distribution.pdf')

### Convert categorical data to numerical

In [38]:
cleanup_enums = {
    "Sex": { "F": 0, "M": 1 },
    "BP": {"LOW": 0, "NORMAL": 1, "HIGH": 2},
    "Cholesterol": {"LOW": 0, "NORMAL": 1, "HIGH": 2},
#     "Drug": {"drugA": 0, "drugB": 1, "drugC": 3, "drugX": 4, "drugY": 5}
}

drug_dataset = drug_dataset.replace(cleanup_enums)

print(drug_dataset.head())

   Age  Sex  BP  Cholesterol  Na_to_K   Drug
0   23    0   2            2   25.355  drugY
1   47    1   0            2   13.093  drugC
2   47    1   0            2   10.114  drugC
3   28    0   1            2    7.798  drugX
4   61    0   0            2   18.043  drugY


### Train/Test Split

In [46]:
from sklearn.model_selection import train_test_split

X = drug_dataset.loc[:, drug_dataset.columns != 'Drug']
Y = drug_dataset['Drug']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20)

### Fitting and Predictions on different Classifiers

In [47]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()

gnb_prediction = gnb.fit(X_train, y_train).predict(X_test)


from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()

dtc_prediction = dtc.fit(X_train, y_train).predict(X_test)


from sklearn.linear_model import Perceptron

per = Perceptron()

per_prediction = per.fit(X_train, y_train).predict(X_test)


from sklearn.neural_network import MLPClassifier

base_mlp = MLPClassifier(hidden_layer_sizes=100)

base_mlp_prediction = base_mlp.fit(X_train, y_train).predict(X_test)

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/tree/tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
